In [1]:
import os
import sys
import pandas as pd
import numpy as np
import networkx as nx
pd.set_option('display.max_colwidth', -1)
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph
import json

In [2]:
from unsupervised_model import UnsupervisedModel

In [3]:
from utils import load_data

In [ ]:
train_data = load_data("example_data/toy-ppi", load_walks=True)

In [4]:
model = UnsupervisedModel(train_prefix="example_data/toy-ppi", model_name="graphsage_mean", validate_iter=10, epochs=1)

In [ ]:
model.train(train_data)

In [5]:
test_data = load_data("example_data/toy-ppi-new")

Removed 0 nodes that lacked proper annotations due to networkx versioning issues.
Loaded data.. now preprocessing..


In [6]:
nodes, val_embeddings = model.predict(test_data, "model_epoch_0.ckpt-3703")

Unexpected missing:  0
9716 train nodes
5041 test nodes
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Restoring trained model.
Model restored.
Computing embeddings...
Saving embeddings...
Embeddings saved.

Timer :: toc --- 9.273179769515991 seconds ---


In [8]:
emb_file = "../../../data/processed/graphsage/example_data/graphsage_mean_small_0.000010/embeddings.npy"
emb_ids_file = "../../../data/processed/graphsage/example_data/graphsage_mean_small_0.000010/embeddings_ids.txt"
new_emb_file = "../../../data/processed/graphsage/example_data/graphsage_mean_small_0.000010/inferred_embeddings.npy"
new_emb_ids_file = "../../../data/processed/graphsage/example_data/graphsage_mean_small_0.000010/inferred_embeddings_ids.txt"

In [9]:
emb = np.load(emb_file)
print(emb)
print(emb.shape)

[[ 0.15025076 -0.17248715 -0.07063117 ... -0.00993037  0.03148542
   0.02569803]
 [-0.04599468 -0.02916748 -0.04853884 ... -0.01297681 -0.00029964
  -0.06876882]
 [ 0.06358294  0.02186082  0.00075023 ...  0.02263598 -0.00043651
  -0.09437178]
 ...
 [-0.06066196 -0.02788442  0.02329348 ... -0.01456614  0.01604309
  -0.00588087]
 [-0.01618305  0.15450895  0.03209545 ... -0.02164444 -0.01092046
   0.11281211]
 [-0.10029155 -0.09989616 -0.06336439 ...  0.00157495  0.03252026
  -0.05727658]]
(14755, 256)


In [10]:
new_emb = np.load(new_emb_file)
print(new_emb)
print(new_emb.shape)

[[-0.06019286 -0.13390696  0.00391163 ... -0.03763382 -0.0718819
  -0.02182361]
 [ 0.03918875  0.06613964 -0.10629884 ...  0.00787869 -0.04075278
  -0.02543417]
 [-0.09038503 -0.1272804   0.03273685 ... -0.05838298 -0.08119658
  -0.05633065]
 ...
 [-0.03726875 -0.04071059 -0.00738414 ... -0.0412822  -0.09551445
  -0.09726841]
 [ 0.02288852  0.01376288  0.05874708 ... -0.05211926 -0.07000946
  -0.07830595]
 [-0.0245814   0.04074003 -0.05222633 ... -0.02351321 -0.01082102
  -0.01466074]]
(14757, 256)


In [11]:
graph_file = "../../../data/interim/graphsage/example_data/toy-ppi-G.json"
new_graph_file = "../../../data/interim/graphsage/example_data/toy-ppi-new-G.json"

In [12]:
G = json_graph.node_link_graph(json.load(open(graph_file)))
ids = [n for n in G.nodes()]
print(len(ids))

14755


In [13]:
G_new = json_graph.node_link_graph(json.load(open(new_graph_file)))
new_ids = [n for n in G_new.nodes()]
print(len(new_ids))

14757


In [14]:
# Map embeddings to node ids
emb_ids = {}
with open(emb_ids_file) as f:
    for i, line in enumerate(f):
        emb_ids[int(line.strip())] = i
embeds = emb[[emb_ids[id] for id in ids]]

In [15]:
# Map embeddings to node ids
new_emb_ids = {}
with open(new_emb_ids_file) as f:
    for i, line in enumerate(f):
        new_emb_ids[int(line.strip())] = i
new_embeds = new_emb[[new_emb_ids[id] for id in new_ids]]

In [16]:
embeds

array([[-0.11521226, -0.05797261, -0.06476892, ..., -0.03636926,
         0.00909875,  0.00033172],
       [-0.01085342, -0.05759057, -0.00472542, ..., -0.01823915,
         0.04191828,  0.10934415],
       [ 0.00469766, -0.01993591, -0.06180527, ...,  0.03038587,
         0.08637361, -0.1176837 ],
       ...,
       [-0.0241402 , -0.11023706, -0.05878201, ...,  0.0071711 ,
        -0.01970433,  0.00668588],
       [ 0.06183387, -0.02834268,  0.01522614, ...,  0.05594001,
        -0.07393727,  0.01290714],
       [-0.06280149, -0.01024674, -0.00924229, ...,  0.00953884,
        -0.04215141,  0.09737176]], dtype=float32)

In [17]:
new_embeds

array([[-0.11488912,  0.13183136,  0.05621745, ..., -0.02232137,
        -0.03558857, -0.02401741],
       [-0.08388648, -0.0319668 ,  0.02802329, ...,  0.02917541,
        -0.0389592 ,  0.00340951],
       [-0.03958556, -0.04796794,  0.09497175, ..., -0.10557486,
        -0.09034798, -0.02041495],
       ...,
       [ 0.04873236, -0.05537066, -0.12041912, ..., -0.04367786,
        -0.13372809, -0.04717691],
       [-0.10992467,  0.11578868,  0.04008877, ..., -0.01996062,
        -0.0328051 , -0.01761653],
       [ 0.01308982,  0.03546999, -0.01608282, ..., -0.06109187,
        -0.10189142, -0.05107431]], dtype=float32)

In [18]:
len(embeds), len(new_embeds)

(14755, 14757)

In [19]:
embeds[7240]

array([ 0.15025076, -0.17248715, -0.07063117, -0.04533799, -0.11264102,
       -0.03991229,  0.03574724,  0.02722474, -0.05216232,  0.08484337,
       -0.08186133, -0.04466963,  0.02381883,  0.04898812, -0.02662823,
        0.10934959, -0.08327614,  0.10564869,  0.07193958,  0.16394489,
        0.0079179 , -0.02374658, -0.05368132,  0.04203405, -0.10707135,
       -0.16801718, -0.00165623, -0.28180903,  0.09020774,  0.0222379 ,
       -0.00354364,  0.03397072,  0.12353374,  0.03380879, -0.07775976,
       -0.02027869,  0.1658048 , -0.00945379, -0.02904884,  0.0223648 ,
       -0.02559132,  0.08652093, -0.05629961,  0.11208373, -0.01479916,
       -0.07531065, -0.05077746,  0.15484314,  0.02203694, -0.00661149,
       -0.00906268, -0.11587612,  0.01699589, -0.15423158, -0.06796177,
        0.02981709,  0.02226621,  0.04680903, -0.02316563, -0.0957397 ,
        0.09411009,  0.01071676,  0.11553867,  0.07314626,  0.13671057,
       -0.2781994 ,  0.0205307 ,  0.04450357, -0.03079117, -0.05

In [20]:
new_embeds[7240]

array([-3.37306820e-02, -3.62402909e-02,  1.14248909e-01,  1.05178125e-01,
       -8.32433626e-02,  6.53367257e-03,  1.18210100e-01,  7.73093700e-02,
       -4.30721901e-02,  7.07504675e-02, -5.98973967e-02,  4.39235941e-02,
        3.37432674e-03,  3.16226631e-02,  1.47987427e-02,  1.18583135e-01,
        3.69802564e-02, -9.20034349e-02,  1.42314071e-02, -1.22065209e-01,
        3.05354223e-02,  1.88357998e-02,  9.46951583e-02,  2.49245763e-03,
        3.81108522e-02,  2.73822527e-02,  2.05185175e-01,  3.67145501e-02,
       -5.36920652e-02, -1.16031557e-01,  1.16465688e-01,  6.66747466e-02,
        4.15463597e-02,  3.31903957e-02, -1.80912539e-02, -5.53578176e-02,
        2.19410937e-02, -7.14422911e-02, -1.08060062e-01,  1.28061593e-01,
        3.40264216e-02, -6.49364199e-03, -6.98352084e-02,  1.96066312e-02,
        3.55742872e-02,  3.92697081e-02,  1.62622631e-02, -7.30516985e-02,
       -5.79127716e-03, -5.32534197e-02, -2.73779500e-02,  2.71662716e-02,
        8.36097375e-02, -

In [21]:
embeds.shape

(14755, 256)

In [22]:
embeds_normed = embeds / np.linalg.norm(embeds, keepdims=True)
new_embeds_normed = new_embeds / np.linalg.norm(new_embeds, keepdims=True)

In [23]:
# np.flip((embeds_normed @ new_embeds_normed[:-2].T).argsort(axis=1), axis=1)[:10]
np.flip((embeds_normed @ new_embeds_normed.T).argsort(axis=1), axis=1)[:10]

array([[12597,  5476, 10562, ..., 14496, 12621, 11512],
       [12108,  8423, 14678, ..., 14309,   982,  5959],
       [12836, 11630, 13589, ..., 13977, 10119, 12249],
       ...,
       [12972,  1446,  7871, ...,   175, 13586,  6830],
       [10841,  5154,  8901, ..., 12479,  9970,  6852],
       [ 2598, 12913,  8944, ...,  3734, 12075,   538]], dtype=int64)

In [24]:
new_embeds_normed[0] @ embeds_normed[0]

4.014888e-06

In [ ]:
new_embeds.sum(1)

#### Run trained model

In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
from tensorflow.python import pywrap_tensorflow
reader = pywrap_tensorflow.NewCheckpointReader('../../../data/processed/graphsage/example_data/graphsage_mean_small_0.000010/model_epoch_0.ckpt-3703')
var_to_shape_map = reader.get_variable_to_shape_map()
for key in var_to_shape_map:
    print("tensor_name: ", key)
    print(reader.get_tensor(key))

In [ ]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

In [ ]:
print_tensors_in_checkpoint_file(file_name='../../../data/processed/graphsage/example_data/graphsage_mean_small_0.000010/model_epoch_0.ckpt-3703'
                                 , all_tensors=False, tensor_name='meanaggregator_1_vars/self_weights')

In [ ]:
# delete the current graph
tf.compat.v1.reset_default_graph()

# import the graph from the file
imported_graph = tf.compat.v1.train.import_meta_graph(
    '../../../data/processed/graphsage/example_data/graphsage_mean_small_0.000010/model_epoch_1.ckpt-7406.meta')

# # list all the tensors in the graph
# for tensor in tf.compat.v1.get_default_graph().get_operations():
#     print(tensor.name)

In [ ]:
graph = tf.compat.v1.get_default_graph()